This is a temporary set of tests.

In [1]:
from quantumgraph import QuantumGraph, ExpectationValue
import numpy as np

An IBMQ account could not be loaded


Set a tolerance for how close the numbers generated by `QuantumGraph` should be to the correct values.

In [2]:
eps = 0.8  # Tolerance for numerical errors

Whether to test using expectation value simulator.

In [3]:
test_ev = False

Tests are done on a graph of 5 qubits

In [4]:
n = 5  # Number of qubits

if test_ev:
    device = ExpectationValue(n)
    graph = QuantumGraph(n,device=device)
else:
    graph = QuantumGraph(n)

First we check that all qubits are indeed in the $\langle Z \rangle = 1$ state.

In [5]:
initial_rho = {'X': 0.0, 'Y': 0.0, 'Z': 1.0}
for j in range(n):
    print(f"Qubit {j} Bloch vector: {graph.get_bloch(j)}")
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)
        print(f"Qubit {j} - Pauli {pauli}: {rho[pauli]}")

Qubit 0 Bloch vector: {'X': np.float64(-0.004730466470197406), 'Y': np.float64(-6.510462315493127e-05), 'Z': np.float64(0.9927610073620987)}
Qubit 0 - Pauli X: -0.004730466470197406
Qubit 0 - Pauli Y: -6.510462315493127e-05
Qubit 0 - Pauli Z: 0.9927610073620987
Qubit 1 Bloch vector: {'X': np.float64(-0.0037121210556256214), 'Y': np.float64(-0.003572469735038541), 'Z': np.float64(0.993357781063)}
Qubit 1 - Pauli X: -0.0037121210556256214
Qubit 1 - Pauli Y: -0.003572469735038541
Qubit 1 - Pauli Z: 0.993357781063
Qubit 2 Bloch vector: {'X': np.float64(-0.0012522009202719524), 'Y': np.float64(-0.003822172691524359), 'Z': np.float64(0.9938735025885583)}
Qubit 2 - Pauli X: -0.0012522009202719524
Qubit 2 - Pauli Y: -0.003822172691524359
Qubit 2 - Pauli Z: 0.9938735025885583
Qubit 3 Bloch vector: {'X': np.float64(-0.000323890575043337), 'Y': np.float64(0.00353969088505579), 'Z': np.float64(0.9933022061505967)}
Qubit 3 - Pauli X: -0.000323890575043337
Qubit 3 - Pauli Y: 0.00353969088505579
Qubi

Now set qubit 2 to the $\langle X \rangle = 1$ state, check that it works and also that the rest are still in the $\langle Z \rangle = 1$ state.

In [6]:
rhos = {}
rho_2 = {'X': 1.0}
graph.set_bloch(rho_2,2)
print(f"Qubit 2 Bloch vector after set_bloch: {graph.get_bloch(2)}")
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        else:
            assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

set_bloch called for qubit 2 with target_expect: {'X': 1.0}, fraction: 1, update: True
Current Bloch vector for qubit 2: {'X': np.float64(-0.0012522009202719524), 'Y': np.float64(-0.003822172691524359), 'Z': np.float64(0.9938735025885583)}
Target Bloch vector for qubit 2: {'X': 1.0, 'Y': 0, 'Z': 0}
Computed unitary matrix U for qubit 2:
[[ 0.70665989-0.00135966j -0.70755078+0.00135966j]
 [ 0.70755078+0.00135966j  0.70665989+0.00135966j]]
Fractional unitary matrix U for qubit 2 (fraction=1):
[[ 0.70665989-0.00135966j -0.70755078+0.00135966j]
 [ 0.70755078+0.00135966j  0.70665989+0.00135966j]]
Euler angles for qubit 2: theta=1.5720562366272164, phi=0.003845714609407149, lambda=2.4226301306060852e-06
Tomography updated after applying set_bloch.
Qubit 2 Bloch vector after set_bloch: {'X': np.float64(0.9936476409146473), 'Y': np.float64(0.003476105945614441), 'Z': np.float64(-0.0032740360981588328)}


for j in range(n):
    print(f"Qubit {j} Bloch vector: {graph.get_bloch(j)}")

The relationship between qubit 2 and any other should now have $\langle ZX \rangle = 1$ (or $\langle XZ \rangle = 1$ depending on order. Check that this is true.

In [7]:
for j in range(n):
    if j!=2:
        assert abs(graph.get_relationship(2,j)['XZ']-1)<eps
        assert abs(graph.get_relationship(j,2)['ZX']-1)<eps

# for j in range(n):
#     if j != 2:
#         # Print the relationship values for debugging
#         rel_2j = graph.get_relationship(2, j)
#         rel_j2 = graph.get_relationship(j, 2)
#         print(f"Relationship (2, {j}): {rel_2j}")
#         print(f"Relationship ({j}, 2): {rel_j2}")
        
#         # Assert the relationships
#         assert abs(rel_2j['XZ'] - 1) < eps, f"Problem with relationship (2, {j}): {rel_2j}"
#         assert abs(rel_j2['ZX'] - 1) < eps, f"Problem with relationship ({j}, 2): {rel_j2}"
# for j in range(n):
#     if j != 2:
#         bloch2 = graph.get_bloch(2)
#         blochj = graph.get_bloch(j)
#         rel = graph.get_relationship(2, j)
#         expected = bloch2['X'] * blochj['Z']
#         print(f"Relationship (2,{j}):", rel)
#         assert abs(rel.get('XZ', 0.0) - expected) < eps

Set qubit 1 to the $\langle Y \rangle=1$ state (without updating the tomography), and then set qubit 3 to the $\langle X \rangle = \langle Z \rangle$ and $\langle Z \rangle=0$ state (with the update). Check that they are correct, and that the others are still what they should be too.

In [8]:
rho_1 = {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
rho_3 = {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
graph.set_bloch(rho_1, 1, update=False)
graph.set_bloch(rho_3, 3)
rho = graph.get_bloch(3)
print(f"Qubit 3 Bloch vector after set_bloch: {rho}")

rho_3 = {'X': 1/np.sqrt(2), 'Y': 1/np.sqrt(2), 'Z': 0.0}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==1:
            assert abs(rho[pauli]-rho_1[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==3:
            print(f"Qubit {j} - Actual: {rho[pauli]}, Expected: {rho_3[pauli]}")
            assert abs(rho[pauli]-rho_3[pauli])<eps, print('Problem with qubit',j,':',rho)
        else:
            assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

set_bloch called for qubit 1 with target_expect: {'X': 0.0, 'Y': 1.0, 'Z': 0.0}, fraction: 1, update: False
Current Bloch vector for qubit 1: {'X': np.float64(-0.002727517637883781), 'Y': np.float64(-0.003663746752808536), 'Z': np.float64(0.9929899222549753)}
Target Bloch vector for qubit 1: {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
Computed unitary matrix U for qubit 1:
[[ 0.70580045+0.00097112j -0.00097112+0.70840938j]
 [ 0.00097112+0.70840938j  0.70580045-0.00097112j]]
Fractional unitary matrix U for qubit 1 (fraction=1):
[[ 0.70580045+0.00097112j -0.00097112+0.70840938j]
 [ 0.00097112+0.70840938j  0.70580045-0.00097112j]]
Euler angles for qubit 1: theta=1.5744859073484543, phi=1.5680495609745155, lambda=-1.5708013940107088
set_bloch called for qubit 3 with target_expect: {'X': 0.0, 'Y': 1.0, 'Z': 0.0}, fraction: 1, update: True
Current Bloch vector for qubit 3: {'X': np.float64(-0.001675646586260877), 'Y': np.float64(-0.005494893374471508), 'Z': np.float64(0.9938799645385439)}
Target Bloch ve

Do half of the rotation of qubit 4 from the $\langle Z \rangle=1$ state to $\langle X \rangle=-1$.

In [9]:
rho_4 = {'X': -1.0, 'Y': 0.0, 'Z': 0.0}
graph.set_bloch(rho_4,4,fraction=0.5)
rho_4 = {'X': -1/np.sqrt(2), 'Y': 0.0, 'Z': 1/np.sqrt(2)}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==1:
            assert abs(rho[pauli]-rho_1[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==3:
            assert abs(rho[pauli]-rho_3[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==4:
            assert abs(rho[pauli]-rho_4[pauli])<eps, print('Problem with qubit',j,':',rho)

set_bloch called for qubit 4 with target_expect: {'X': -1.0, 'Y': 0.0, 'Z': 0.0}, fraction: 0.5, update: True
Current Bloch vector for qubit 4: {'X': np.float64(-0.005123980256025431), 'Y': np.float64(-0.0020547599995474233), 'Z': np.float64(0.993532999456762)}
Target Bloch vector for qubit 4: {'X': -1.0, 'Y': 0.0, 'Z': 0.0}
Computed unitary matrix U for qubit 4:
[[ 0.70892742+0.00073119j  0.70528068+0.00073119j]
 [-0.70528068+0.00073119j  0.70892742-0.00073119j]]
Fractional unitary matrix U for qubit 4 (fraction=0.5):
[[ 0.92437206+0.0003955j  0.38149178+0.0003955j]
 [-0.38149178+0.0003955j  0.92437206-0.0003955j]]
Euler angles for qubit 4: theta=0.7828196164720663, phi=3.140128057861255, lambda=-3.140983784662981
Tomography updated after applying set_bloch.


Various tests for two qubits.

In [10]:
relationships = {'ZZ':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)

relationships = {'XX':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)
    
relationships = {'ZX':+1,'XZ':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)

### Additional Tests for qubit ordering in `set_relationship`

In [11]:
relationships = {'XX':+1}
graph.set_relationship(relationships,0,1)
rho_01 = graph.get_relationship(0,1)
for (pauli,sign) in relationships.items():
    assert abs(rho_01[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_01)

In [12]:
print(rho_01)

{'XX': np.float64(0.9909552061255673), 'XY': np.float64(0.002338813006651442), 'XZ': np.float64(-0.00495712560524509), 'YX': np.float64(0.003666444635346955), 'YY': np.float64(-0.0007234271456779962), 'YZ': np.float64(-0.00244939221803704), 'ZX': np.float64(-0.00289622451153272), 'ZY': np.float64(-0.0013714461717840819), 'ZZ': np.float64(0.0004981342763795416)}
